In [7]:
from pcxarray import pc_query, prepare_timeseries
from pcxarray.utils import load_census_shapefile
from distributed import Client # dask distributed client
from tqdm import tqdm
import xarray as xr
import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib.animation import FuncAnimation

In [ ]:
# client = Client(n_workers=10, memory_limit='2GB')
# display(client)

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 10,Total memory: 18.63 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:49547,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:49572,Total threads: 1
Dashboard: http://127.0.0.1:49577/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:49550,


2025-06-28 19:48:20,113 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:49576' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('open_rasterio-original-getitem-fafa1bb9fb2cdd6bc6a0246f24f393ee', 0, 0, 0), 'original-open_rasterio-f3ed32fe124b824781fb93e1ce118592<this-array>-b5f9e5fa2a77413d161b905696884849', ('isnan-4caee0930bdaa61c454ffeb64d69149d', 2, 0, 0)} (stimulus_id='handle-worker-cleanup-1751158100.1132889')
2025-06-28 19:48:20,128 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:49587' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('isnan-aa7cba3ccb84eaca3272f110a29c03a5', 2, 0, 1), 'original-open_rasterio-869c5eb506e0fa57038c898ee29f2dab<this-array>-23b08febdcb1acf974f72909a65148c9', ('isnan-aa7cba3ccb84eaca3272f110a29c03a5', 1, 0, 1)} (stimulus_id='handle-worker-cleanup-1751158100.128024')
2025-06-28 19:48:20,130 - distributed.scheduler - WARNING - Re

In [9]:
counties_gdf = load_census_shapefile(level="county")
okt_gdf = counties_gdf[counties_gdf["NAME"] == "Oktibbeha"] # Oktibbeha County, MS
okt_gdf = okt_gdf.to_crs(32616) # UTM 16N

selected_geom = okt_gdf.geometry.union_all()

In [10]:
items_gdf = pc_query(
    collections=["sentinel-2-l2a"],
    geometry=selected_geom,
    datetime='2024', # grab all of 2024
    crs=okt_gdf.crs,
)

In [ ]:
da = prepare_timeseries(
    items_gdf=items_gdf,
    geometry=selected_geom,
    crs=okt_gdf.crs,
    bands=["B02", "B04", "B08", "SCL"], # Blue, Red, NIR, Scene Classification Layer
    # chunks={"x": 4096, "y": 4096},
    masked=True,
    max_workers=-1, # setting max_workers to -1 will use the number of available CPU cores
)

Processing items:   1%|          | 1/134 [04:41<10:23:25, 281.25s/timestep]
2025-06-28 19:48:20,108 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('open_rasterio-getitem-db8b71ecd82775e3501164789c9c9b84', 0, 0, 0))" coro=<Worker.execute() done, defined at /Users/dak/projects/pcxarray/.venv/lib/python3.12/site-packages/distributed/worker_state_machine.py:3607>> ended with CancelledError
2025-06-28 19:48:20,108 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('open_rasterio-getitem-c859c87f28f99e4d79f09d3d616f6fea', 0, 0, 0))" coro=<Worker.execute() done, defined at /Users/dak/projects/pcxarray/.venv/lib/python3.12/site-packages/distributed/worker_state_machine.py:3607>> ended with CancelledError
2025-06-28 19:48:20,108 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('open_rasterio-getitem-11af986ca709f7e06964ec7509594966', 0, 0, 1))"

KeyboardInterrupt: 

In [ ]:
# now, create a monthly composite using the median
monthly_composite = da.resample(datetime='1M').median()

fig, ax = plt.subplots(figsize=(10, 10))
def update(frame):
    ax.clear()
    (monthly_composite.isel(datetime=frame) / 5000).plot.imshow(ax=ax)
    ax.set_title(f"Month: {monthly_composite['datetime'][frame].dt.strftime('%Y-%m').values}")
    ax.set_xlabel(None)
    ax.set_ylabel(None)
    ax.set_xticks([])
    ax.set_yticks([])

ani = FuncAnimation(fig, update, frames=len(monthly_composite['datetime']), repeat=False)

# show in notebook
HTML(ani.to_jshtml(fps=1, default_mode='once')) 

KeyError: 'datetime'

In [ ]:
for datetime, group in tqdm(items_gdf.groupby("properties.datetime"), desc="Preparing timesteps"):
    break

Preparing timesteps:   0%|          | 0/134 [00:00<?, ?it/s]


In [ ]:
from planetary_computer import sign_url
import rioxarray as rxr

# test_itm = group.iloc[1]  # Get the first item in the group
# for i in range(len(items_gdf)):
test_itm = items_gdf.iloc[0]  # Get the first item in the items_gdf
signed_url = sign_url(test_itm['assets.B04.href'])
test_image = rxr.open_rasterio(signed_url)

test_image.rio.transform(recalc=True)
test_image.rio.resolution()

NameError: name 'items_gdf' is not defined

(10.0, -10.0)

In [ ]:
test

NameError: name 'test_itm' is not defined